In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
cd ..

/Users/yunxizzy/code/theresa-sporn/WorkforceSentimentMonitoring


In [5]:
!ls

MANIFEST.in                           notebooks
Makefile                              raw_data
README.md                             requirements.txt
WorkforceSentimentMonitoring          scripts
WorkforceSentimentMonitoring.egg-info setup.py
app.py                                test_data
build                                 tests
dist


In [6]:
from WorkforceSentimentMonitoring.data import get_data, merge, holdout, encode_target, \
drop_wrong_language, get_prepaired_data

In [132]:
from WorkforceSentimentMonitoring.feature_engineering import SCORE_COLS, FEATURE_COLS, minmax_when_needed

In [8]:
from WorkforceSentimentMonitoring.encoders import Preprocessor, FeatureEngineer, CustomMinMaxScaler, PredictionFeaturesExtractor

# Load Data

## Try with one text column

### Prepare train test sets

In [9]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Dropping initial text columns...
Identifying entries in other languages...
Drop 437 entries? [y] / n
y
Dropping 437 entries...
Process completed.
Splitting train and test...
Encoding targets...
Done!


In [10]:
display(X_train.head())
display(y_train.head())

,review
0,"Good place to workenvironment, colleagues, opp..."
1,"Great place to workVery smart people, focused ..."
2,Great opportunities to develop skills in diffe...
3,working at applegreat pay! great benefits and ...
4,This place is not good.There is nothing good a...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,2,1,2,2,1,1
1,1,2,2,2,2,2
2,2,2,1,1,1,1
3,2,2,2,2,2,2
4,0,0,0,0,0,1


In [101]:
X_train.isnull().sum()

summary              62
positives             0
negatives             1
advice_to_mgmt    15691
review                0
dtype: int64

In [102]:
X_test.isnull().sum()

summary             18
positives            0
negatives            0
advice_to_mgmt    6748
review               0
dtype: int64

In [17]:
import pandas as pd

In [18]:
X = pd.concat([X_train, X_test])

In [19]:
y = pd.concat([y_train, y_test])

In [20]:
display(X.head())
display(y.head())

,summary,positives,negatives,advice_to_mgmt,review
0,Pathways Operations Manager,A very fast paced and customer focused company,Not a very employee friendly company,NaN,Pathways Operations Manager A very fast paced ...
1,Ok for short term work,Benefits from 1st day of employment,"Physically demanding, long shifts, high produc...",NaN,Ok for short term work Benefits from 1st day o...
2,Great People,Work with people who are passionate about what...,"Need open offices, many offices don't have any...",Encourage people to move between teams. Breadt...,Great People Work with people who are passiona...
3,Constantly making history,Great work culture Very Direct and no politick...,Constant change of plans Work life Balance,NaN,Constantly making history Great work culture V...
4,What you would expect,Pay is good for job being done,"retail hours, getting time off",NaN,What you would expect Pay is good for job bein...


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,0,2,2,2,2,2
1,0,1,1,2,1,1
2,2,2,2,2,2,2
3,1,2,2,2,2,2
4,1,2,1,2,2,2


In [66]:
df = pd.concat([X, y], axis=1)

In [67]:
df.dropna(inplace=True)

In [93]:
df_copy = df.copy()

In [98]:
df_copy.to_csv('raw_data/pos_neg_df.csv', encoding='utf-8', index=False)

### Features engineering

In [21]:
from WorkforceSentimentMonitoring.feature_engineering import get_lengths,\
get_linear_regression_cols, get_subjectivity_polarity_columns,\
getPolarity, getSubjectivity, add_multinomial_nb_prediction_feature

In [111]:
X_train_copy = X_train.copy()
y_train_copy = y_train.copy()
X_test_copy = X_test.copy()
y_test_copy = y_test.copy()

In [112]:
pre = Preprocessor()
X_train_copy_pre = preprocessor.fit_transform(X_train_copy)
X_test_copy_pre = preprocessor.fit_transform(X_test_copy)

In [113]:
fea = FeatureEngineer()
X_train_copy_fea = feature_engineering.fit_transform(X_train_copy_pre)
X_test_copy_fea = feature_engineering.fit_transform(X_test_copy_pre)

100%|██████████| 1/1 [00:12<00:00, 12.26s/it]


In [114]:
X_train_copy_fea

,review,review_length,subjectivity_review,polarity_review
0,good place workenvironment colleague opportuni...,110,0.350000,0.350000
1,great place workvery smart people focused grea...,478,0.649405,0.217092
2,great opportunity develop skill different area...,323,0.450000,0.387500
3,working applegreat pay great benefit support a...,181,0.683333,0.466667
4,place good nothing good placezero work life ba...,123,0.433333,0.466667
...,...,...,...,...
36659,analystamazon fast paced exciting place work l...,173,0.608000,0.232000
36660,use okapple use great company changed last yea...,189,0.469444,0.300000
36661,cloud support engineeramazon leadership princi...,188,0.200000,0.100000
36662,fulfillment associategreat pay benefit positiv...,89,0.545455,0.227273


In [115]:
X_test_copy_fea

,review,review_length,subjectivity_review,polarity_review
0,working amazon call centergreat environment am...,376,0.820833,0.691667
1,microsoft student partner programgood communit...,217,0.266667,0.020000
2,think long hardplenty opportunity abound howev...,527,0.623077,0.106410
3,limitless opportunity intelligent senior manag...,667,0.467727,0.269318
4,superawesome job friendly environment salary g...,205,0.586000,0.439000
...,...,...,...,...
15709,senior program managerimpressive work great br...,327,0.310000,0.080000
15710,take yr understand take get thing done microso...,865,0.427846,-0.015979
15711,bad jobif least average work ethnic get along ...,367,0.546329,0.167857
15712,amazonno pro work hard temp never b hiredhard ...,108,0.541667,-0.291667


In [117]:
df_train = pd.concat([X_train_copy_fea, y_train_copy], axis=1)
df_test = pd.concat([X_test_copy_fea, y_test_copy], axis=1)

In [119]:
df_train

,review,review_length,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,good place workenvironment colleague opportuni...,110,0.350000,0.350000,2,1,2,2,1,1
1,great place workvery smart people focused grea...,478,0.649405,0.217092,1,2,2,2,2,2
2,great opportunity develop skill different area...,323,0.450000,0.387500,2,2,1,1,1,1
3,working applegreat pay great benefit support a...,181,0.683333,0.466667,2,2,2,2,2,2
4,place good nothing good placezero work life ba...,123,0.433333,0.466667,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
36659,analystamazon fast paced exciting place work l...,173,0.608000,0.232000,2,2,2,2,2,2
36660,use okapple use great company changed last yea...,189,0.469444,0.300000,1,1,0,1,0,0
36661,cloud support engineeramazon leadership princi...,188,0.200000,0.100000,2,2,2,2,2,2
36662,fulfillment associategreat pay benefit positiv...,89,0.545455,0.227273,2,2,2,2,2,2


In [120]:
df_test

,review,review_length,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,working amazon call centergreat environment am...,376,0.820833,0.691667,2,2,2,2,2,2
1,microsoft student partner programgood communit...,217,0.266667,0.020000,2,2,2,2,1,1
2,think long hardplenty opportunity abound howev...,527,0.623077,0.106410,0,0,0,2,0,1
3,limitless opportunity intelligent senior manag...,667,0.467727,0.269318,2,2,2,2,2,2
4,superawesome job friendly environment salary g...,205,0.586000,0.439000,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
15709,senior program managerimpressive work great br...,327,0.310000,0.080000,0,2,0,2,0,0
15710,take yr understand take get thing done microso...,865,0.427846,-0.015979,2,0,1,2,0,0
15711,bad jobif least average work ethnic get along ...,367,0.546329,0.167857,1,1,2,2,1,2
15712,amazonno pro work hard temp never b hiredhard ...,108,0.541667,-0.291667,0,0,0,0,0,0


In [121]:
df_train_nb, df_test_nb, scores_dict = add_multinomial_nb_prediction_feature(df_train, df_test, y_train_copy, y_test_copy)

In [122]:
scores_dict

{'work-balance': {'review': 0.4964999363624793},
 'culture-values': {'review': 0.654320987654321},
 'career-opportunities': {'review': 0.6010563828433244},
 'comp-benefits': {'review': 0.7145220822196767},
 'senior-mgmt': {'review': 0.5043273514063892},
 'overall': {'review': 0.5154639175257731}}

In [140]:
X_train_nb = df_train_nb.drop(columns=SCORE_COLS)
X_test_nb = df_test_nb.drop(columns=SCORE_COLS)

In [128]:
#sca = CustomMinMaxScaler()

In [141]:
X_train_nb

,review,review_length,subjectivity_review,polarity_review,review_work-balance_nb,review_culture-values_nb,review_career-opportunities_nb,review_comp-benefits_nb,review_senior-mgmt_nb,review_overall_nb
0,good place workenvironment colleague opportuni...,110,0.350000,0.350000,2,2,2,2,2,2
1,great place workvery smart people focused grea...,478,0.649405,0.217092,2,2,2,2,2,2
2,great opportunity develop skill different area...,323,0.450000,0.387500,2,2,2,2,2,2
3,working applegreat pay great benefit support a...,181,0.683333,0.466667,2,2,2,2,2,2
4,place good nothing good placezero work life ba...,123,0.433333,0.466667,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
36659,analystamazon fast paced exciting place work l...,173,0.608000,0.232000,2,2,2,2,2,2
36660,use okapple use great company changed last yea...,189,0.469444,0.300000,2,2,2,2,2,2
36661,cloud support engineeramazon leadership princi...,188,0.200000,0.100000,2,2,2,2,2,2
36662,fulfillment associategreat pay benefit positiv...,89,0.545455,0.227273,2,2,2,2,2,2


In [139]:
X_test_nb

,review,review_length,subjectivity_review,polarity_review,review_work-balance_nb,review_culture-values_nb,review_career-opportunities_nb,review_comp-benefits_nb,review_senior-mgmt_nb,review_overall_nb
0,working amazon call centergreat environment am...,376,0.820833,0.691667,2,2,2,2,2,2
1,microsoft student partner programgood communit...,217,0.266667,0.020000,2,2,2,2,2,2
2,think long hardplenty opportunity abound howev...,527,0.623077,0.106410,2,2,2,2,2,2
3,limitless opportunity intelligent senior manag...,667,0.467727,0.269318,2,2,2,2,2,2
4,superawesome job friendly environment salary g...,205,0.586000,0.439000,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...
15709,senior program managerimpressive work great br...,327,0.310000,0.080000,2,2,2,2,2,2
15710,take yr understand take get thing done microso...,865,0.427846,-0.015979,2,2,2,2,2,2
15711,bad jobif least average work ethnic get along ...,367,0.546329,0.167857,2,2,2,2,2,2
15712,amazonno pro work hard temp never b hiredhard ...,108,0.541667,-0.291667,2,2,2,2,2,2


In [137]:
#X_train_scaled, X_test_scaled = minmax_when_needed(X_train_nb, X_test_nb)

In [142]:
#X_train_scaled, X_test_scaled = sca.fit_transform(X_train_nb, X_test_nb)

In [144]:
X_train_fin = X_train_nb.drop(columns='review')

In [146]:
X_test_fin = X_test_nb.drop(columns='review')

In [147]:
X_train_lin, X_test_lin = get_linear_regression_cols(X_train_fin, X_test_fin, y_train_copy)

100%|██████████| 6/6 [00:00<00:00, 38.13it/s]


In [148]:
X_train_lin

,review_length,subjectivity_review,polarity_review,review_work-balance_nb,review_culture-values_nb,review_career-opportunities_nb,review_comp-benefits_nb,review_senior-mgmt_nb,review_overall_nb,work-balance_regression,culture-values_regression,career-opportunities_regression,comp-benefits_regression,senior-mgmt_regression,overall_regression
0,110,0.350000,0.350000,2,2,2,2,2,2,1.375650,1.606042,1.529187,1.680241,1.354517,1.383911
1,478,0.649405,0.217092,2,2,2,2,2,2,1.125351,1.370052,1.336546,1.579053,1.132092,1.240791
2,323,0.450000,0.387500,2,2,2,2,2,2,1.331323,1.558131,1.491874,1.671894,1.313702,1.370073
3,181,0.683333,0.466667,2,2,2,2,2,2,1.411935,1.674873,1.598713,1.741391,1.445444,1.459033
4,123,0.433333,0.466667,2,2,2,2,2,2,1.456840,1.693431,1.606986,1.737406,1.449964,1.458981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36659,173,0.608000,0.232000,2,2,2,2,2,2,1.233026,1.493444,1.440189,1.628354,1.252984,1.308699
36660,189,0.469444,0.300000,2,2,2,2,2,2,1.298483,1.539203,1.475543,1.651154,1.293038,1.342153
36661,188,0.200000,0.100000,2,2,2,2,2,2,1.166916,1.380157,1.330546,1.543776,1.113174,1.203405
36662,89,0.545455,0.227273,2,2,2,2,2,2,1.261187,1.519951,1.460771,1.635234,1.275698,1.319454


In [149]:
X_test_lin

,review_length,subjectivity_review,polarity_review,review_work-balance_nb,review_culture-values_nb,review_career-opportunities_nb,review_comp-benefits_nb,review_senior-mgmt_nb,review_overall_nb,work-balance_regression,culture-values_regression,career-opportunities_regression,comp-benefits_regression,senior-mgmt_regression,overall_regression
0,376,0.820833,0.691667,2,2,2,2,2,2,1.520343,1.780646,1.694785,1.825779,1.565813,1.569095
1,217,0.266667,0.020000,2,2,2,2,2,2,1.086260,1.307661,1.270230,1.505216,1.042125,1.150134
2,527,0.623077,0.106410,2,2,2,2,2,2,1.024197,1.265171,1.245662,1.518524,1.022628,1.159861
3,667,0.467727,0.269318,2,2,2,2,2,2,1.131172,1.339360,1.305563,1.567136,1.093190,1.228024
4,205,0.586000,0.439000,2,2,2,2,2,2,1.393156,1.643326,1.568755,1.720045,1.407723,1.432289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15709,327,0.310000,0.080000,2,2,2,2,2,2,1.097117,1.314606,1.278002,1.519618,1.053453,1.168308
15710,865,0.427846,-0.015979,2,2,2,2,2,2,0.845938,1.042950,1.050012,1.402081,0.786676,1.006511
15711,367,0.546329,0.167857,2,2,2,2,2,2,1.130084,1.371231,1.333987,1.566896,1.125872,1.226868
15712,108,0.541667,-0.291667,2,2,2,2,2,2,0.836629,1.107051,1.105162,1.388019,0.847545,0.987255


In [150]:
X_train_scaled, X_test_scaled = minmax_when_needed(X_train_lin, X_test_lin)

In [217]:
X_train_scaled_new = X_train_scaled.iloc[:,:9]
X_test_scaled_new = X_test_scaled.iloc[:,:9]

In [152]:
X_test_scaled

,review_length,subjectivity_review,polarity_review,review_work-balance_nb,review_culture-values_nb,review_career-opportunities_nb,review_comp-benefits_nb,review_senior-mgmt_nb,review_overall_nb,work-balance_regression,culture-values_regression,career-opportunities_regression,comp-benefits_regression,senior-mgmt_regression,overall_regression
0,0.027771,0.820833,0.691667,1.0,0.0,0.0,0.0,1.0,0.0,0.909631,0.931481,0.929999,0.911151,0.928431,0.912729
1,0.015155,0.266667,0.020000,1.0,0.0,0.0,0.0,1.0,0.0,0.813583,0.843022,0.836929,0.770661,0.830983,0.777127
2,0.039752,0.623077,0.106410,1.0,0.0,0.0,0.0,1.0,0.0,0.799851,0.835075,0.831543,0.776494,0.827355,0.780275
3,0.050861,0.467727,0.269318,1.0,0.0,0.0,0.0,1.0,0.0,0.823521,0.848951,0.844674,0.797799,0.840486,0.802337
4,0.014203,0.586000,0.439000,1.0,0.0,0.0,0.0,1.0,0.0,0.881489,0.905799,0.902371,0.864812,0.899014,0.868450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15709,0.023883,0.310000,0.080000,1.0,0.0,0.0,0.0,1.0,0.0,0.815986,0.844321,0.838632,0.776974,0.833091,0.783009
15710,0.066571,0.427846,-0.015979,1.0,0.0,0.0,0.0,1.0,0.0,0.760408,0.793515,0.788653,0.725462,0.783449,0.730641
15711,0.027057,0.546329,0.167857,1.0,0.0,0.0,0.0,1.0,0.0,0.823280,0.854911,0.850905,0.797693,0.846567,0.801963
15712,0.006506,0.541667,-0.291667,1.0,0.0,0.0,0.0,1.0,0.0,0.758348,0.805504,0.800743,0.719299,0.794776,0.724409


In [191]:
y_train_copy

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,2,1,2,2,1,1
1,1,2,2,2,2,2
2,2,2,1,1,1,1
3,2,2,2,2,2,2
4,0,0,0,0,0,1
...,...,...,...,...,...,...
36659,2,2,2,2,2,2
36660,1,1,0,1,0,0
36661,2,2,2,2,2,2
36662,2,2,2,2,2,2


### Model

In [218]:
# combine neutral & negatives
def encode_y(y):
    encoding = {0: 0, 0 : 0, 1 : 0, 2 : 1, 2 : 1}
    for col in y.columns:
        y[col] = y[col].map(encoding)
    return y

In [219]:
y_train_copy_new = encode_y(y_train_copy)

In [221]:
y_test_copy_new = encode_y(y_test_copy)

In [220]:
y_train_copy_new

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,0,1,1,0,0
1,0,1,1,1,1,1
2,1,1,0,0,0,0
3,1,1,1,1,1,1
4,0,0,0,0,0,0
...,...,...,...,...,...,...
36659,1,1,1,1,1,1
36660,0,0,0,0,0,0
36661,1,1,1,1,1,1
36662,1,1,1,1,1,1


In [223]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
def train_log(X_train, y_train, X_test, y_test, SCORE_COLS):
    prediction_scores_dict = {}
    for target in tqdm(SCORE_COLS):
        model = LogisticRegression(max_iter=1000)
        model.fit(X_train, y_train[target])
        prediction_scores_dict[target] = model.score(X_test, y_test[target])

    return model, prediction_scores_dict

In [156]:
model, scores = train_log(X_train_scaled, y_train_copy, X_test_scaled, y_test_copy, SCORE_COLS)

100%|██████████| 6/6 [00:14<00:00,  2.36s/it]


In [224]:
_, sc = train_log(X_train_scaled_new, y_train_copy_new, X_test_scaled_new, y_test_copy_new, SCORE_COLS)

100%|██████████| 6/6 [00:00<00:00,  9.31it/s]


In [225]:
sc

{'work-balance': 0.5997836324296806,
 'culture-values': 0.6935217003945526,
 'career-opportunities': 0.6396843578974163,
 'comp-benefits': 0.7149675448644521,
 'senior-mgmt': 0.6118747613592974,
 'overall': 0.5876288659793815}

In [157]:
scores

{'work-balance': 0.5250095456281023,
 'culture-values': 0.6652666412116583,
 'career-opportunities': 0.6099656357388317,
 'comp-benefits': 0.7141402570955836,
 'senior-mgmt': 0.5480463281150566,
 'overall': 0.5262822960417463}

In [184]:
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
def train_RanForest(X_train, y_train, X_test, y_test, SCORE_COLS):
    prediction_scores_dict = {}
    for target in tqdm(SCORE_COLS):
        model = RandomForestClassifier(
                class_weight='balanced',
                max_depth=
                )
        model.fit(X_train, y_train[target])
        prediction_scores_dict[target] = model.score(X_test, y_test[target])

    return model, prediction_scores_dict

In [185]:
_, score_dict = train_RanForest(X_train_scaled, y_train_copy, X_test_scaled, y_test_copy, SCORE_COLS)

100%|██████████| 6/6 [00:57<00:00,  9.61s/it]


In [186]:
score_dict

{'work-balance': 0.4683085147002673,
 'culture-values': 0.6292478045055364,
 'career-opportunities': 0.5579101438207967,
 'comp-benefits': 0.6664757541046201,
 'senior-mgmt': 0.4915998472699504,
 'overall': 0.4578719613083874}

In [231]:
_, sc_dict = train_RanForest(X_train_scaled_new, y_train_copy_new, X_test_scaled_new, y_test_copy_new, SCORE_COLS)

100%|██████████| 6/6 [00:34<00:00,  5.71s/it]


In [232]:
sc_dict

{'work-balance': 0.5707649229985999,
 'culture-values': 0.6581392388952526,
 'career-opportunities': 0.6099019982181494,
 'comp-benefits': 0.682194221713122,
 'senior-mgmt': 0.584510627465954,
 'overall': 0.5560010182003309}

### GridSearch

#### LogReg

In [164]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy import stats

In [233]:
log = LogisticRegression(class_weight='balanced', max_iter=1000)
param = {
    'C': stats.loguniform(0.001, 10),
    'multi_class': ['multinomial', 'ovr', 'auto'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
         }
random_search = RandomizedSearchCV(log, param, scoring='balanced_accuracy', n_iter=100, cv=10, n_jobs=-1)

In [204]:
r_search = random_search.fit(X_train_scaled, y_train_copy['work-balance'])

In [205]:
r_search.best_estimator_

LogisticRegression(C=0.16721425226220143, class_weight='balanced',
                   max_iter=1000, multi_class='multinomial',
                   solver='newton-cg')

In [206]:
r_search.best_params_

{'C': 0.16721425226220143, 'multi_class': 'multinomial', 'solver': 'newton-cg'}

In [207]:
r_search.best_score_

0.43180485865384705

In [235]:
se_old = random_search.fit(X_train_scaled, y_train_copy['work-balance'])

In [236]:
se_old.best_score_, se_old.best_params_

(0.6012205119600132,
 {'C': 0.008710880154955854, 'multi_class': 'auto', 'solver': 'saga'})

In [ ]:
# 2 classes:

In [227]:
r_se = random_search.fit(X_train_scaled_new, y_train_copy_new['work-balance'])

In [228]:
r_se.best_params_, r_se.best_score_

({'C': 0.2715081169730005, 'multi_class': 'multinomial', 'solver': 'sag'},
 0.6006419756342424)

In [239]:
ra_search = random_search.fit(X_train_scaled, y_train_copy['overall'])

In [240]:
ra_search.best_score_, ra_search.best_params_

(0.5803429287805841,
 {'C': 0.0031474043334631502, 'multi_class': 'ovr', 'solver': 'liblinear'})

In [237]:
r_search = random_search.fit(X_train_scaled_new, y_train_copy_new['overall'])

In [238]:
r_search.best_score_, r_search.best_params_

(0.5808365692161532,
 {'C': 0.0010090563254478795, 'multi_class': 'ovr', 'solver': 'liblinear'})

In [242]:
rand_search = random_search.fit(X_train_scaled, y_train_copy['comp-benefits'])
rand_search.best_score_, r_search.best_params_

(0.5934632664595034,
 {'C': 0.0012978171748245116, 'multi_class': 'auto', 'solver': 'liblinear'})

In [241]:
rand_search = random_search.fit(X_train_scaled_new, y_train_copy_new['comp-benefits'])
rand_search.best_score_, r_search.best_params_

(0.593062830830505,
 {'C': 0.0011523041750358537, 'multi_class': 'auto', 'solver': 'saga'})

In [243]:
rand_search = random_search.fit(X_train_scaled, y_train_copy['culture-values'])
rand_search.best_score_, r_search.best_params_

(0.6272770023787134,
 {'C': 0.016493845542484856, 'multi_class': 'auto', 'solver': 'lbfgs'})

In [244]:
rand_search = random_search.fit(X_train_scaled_new, y_train_copy_new['culture-values'])
rand_search.best_score_, r_search.best_params_

(0.6276960269577538,
 {'C': 0.0011114862565525657, 'multi_class': 'multinomial', 'solver': 'saga'})

In [245]:
rand_search = random_search.fit(X_train_scaled, y_train_copy['senior-mgmt'])
rand_search.best_score_, r_search.best_params_

(0.6117806256611934,
 {'C': 0.0012455866202063352, 'multi_class': 'ovr', 'solver': 'liblinear'})

In [246]:
rand_search = random_search.fit(X_train_scaled_new, y_train_copy_new['senior-mgmt'])
rand_search.best_score_, r_search.best_params_

(0.6108695540882858,
 {'C': 0.3636103856240571, 'multi_class': 'ovr', 'solver': 'sag'})

In [247]:
rand_search = random_search.fit(X_train_scaled, y_train_copy['career-opportunities'])
rand_search.best_score_, r_search.best_params_

(0.6045919294527982,
 {'C': 0.003977921048620328, 'multi_class': 'ovr', 'solver': 'lbfgs'})

In [248]:
rand_search = random_search.fit(X_train_scaled_new, y_train_copy_new['career-opportunities'])
rand_search.best_score_, r_search.best_params_

(0.6035629113362464,
 {'C': 0.4357102526484807, 'multi_class': 'auto', 'solver': 'newton-cg'})

#### RandomForest

In [187]:
clf = RandomForestClassifier()
params = {
    'min_samples_split': stats.loguniform(0.001, 1),
    'max_depth': stats.uniform(1,10),
    'min_samples_leaf': stats.loguniform(0.001, 1),
    'class_weight': ['balanced', 'balanced_subsample']
         }
ran_search = RandomizedSearchCV(clf, params, scoring='balanced_accuracy', n_iter=100, cv=5, n_jobs=-1)

In [193]:
search = ran_search.fit(X_train_scaled, y_train_copy['work-balance'])

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [229]:
search = ran_search.fit(X_train_scaled_new, y_train_copy_new['work-balance'])

In [230]:
search.best_params_, search.best_score_

({'class_weight': 'balanced_subsample',
  'max_depth': 8.591824034587917,
  'min_samples_leaf': 0.0010405651490456407,
  'min_samples_split': 0.00928296860705334},
 0.6100161717760527)

### trial: Group in 2 classes

In [129]:
# remap scores into 2 class targets (leave score 3 as '3')
'''def remap(data, dict_labels):
    for key, values in dict_labels.items():
        data.replace({key: {1: '0', 2: '0', 3: '1', 4: '2', 5: '2'}}, inplace=True)
    return data'''

In [114]:
#small_target.apply(lambda row: remap(small_target, small_target), axis=1)

I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping c

I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping 

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balanc

...

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapp

I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-valu

I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping overall
DONE
I am remapping work-balance
I am remapping culture-values
I am remapping career-opportunities
I am remapping comp-benefits
I am remapping senior-mgmt
I am remapping ove

0            work-balance culture-values career-opport...
1            work-balance culture-values career-opport...
2            work-balance culture-values career-opport...
3            work-balance culture-values career-opport...
4            work-balance culture-values career-opport...
                              ...                        
9995         work-balance culture-values career-opport...
9996         work-balance culture-values career-opport...
9997         work-balance culture-values career-opport...
9998         work-balance culture-values career-opport...
9999         work-balance culture-values career-opport...
Length: 10000, dtype: object

In [125]:
#small_target = small_target.astype('uint8')

In [ ]:
#small_target

## Try with all the text columns

### Prepare train test sets

In [11]:
X_train_all, X_test_all, y_train_all, y_test_all = get_prepaired_data(keep_text_cols=True)

Reading data...
Merging data into a single DataFrame...
Identifying entries in other languages...
Drop 441 entries? [y] / n
y
Dropping 441 entries...
Process completed.
Splitting train and test...
Encoding targets...
Done!


In [12]:
display(X_train_all.head())
display(y_train_all.head())

,summary,positives,negatives,advice_to_mgmt,review
0,Many Microsofts within Microsoft,"Smart colleagues, flexible schedule, and good ...",Review model doesn't foster enough collaboration.,Please work to foster more collaboration withi...,Many Microsofts within MicrosoftSmart colleagu...
1,One of the best MNC company in Singapore!,- very responsive welfare which covers all are...,- lacking bonus payout - international interna...,- keep up with the vibrant services given to e...,One of the best MNC company in Singapore!- ver...
2,Business Specialist,An amazing opportunity. So many talented co-wo...,There really isn't anything negative to point ...,Keep up the great work. I learned so much and ...,Business SpecialistAn amazing opportunity. So ...
3,Fast Paced Environment where you learn a lot,"The upward mobility, fast paced, growth and te...",Amazon is the gift that takes. Ensure you have...,Allow your leaders in the fulfillment centers ...,Fast Paced Environment where you learn a lotTh...
4,Overview,"High Salary, Benefits, Raises yearly.",Short Breaks. Unattainable Metrics. Dealing wi...,Treating everyone equally. Don't Play Favorites,"OverviewHigh Salary, Benefits, Raises yearly.S..."


,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,1,1,2,1,1
1,2,2,1,2,1,1
2,2,2,2,1,1,2
3,0,2,2,2,1,2
4,1,0,0,2,1,1


# Prepare 2-class target

In [34]:
# transform scores into 2 classes: 0 => negative; 1 => positive
#target.apply(lambda row: remap(target, target), axis=1)

In [55]:
X_train_all_copy = X_train_all.copy()
y_train_all_copy = y_train_all.copy()
X_test_all_copy = X_test_all.copy()
y_test_all_copy = y_test_all.copy()

In [56]:
X_train_all_copy

,summary,positives,negatives,advice_to_mgmt,review
0,Many Microsofts within Microsoft,"Smart colleagues, flexible schedule, and good ...",Review model doesn't foster enough collaboration.,Please work to foster more collaboration withi...,Many Microsofts within MicrosoftSmart colleagu...
1,One of the best MNC company in Singapore!,- very responsive welfare which covers all are...,- lacking bonus payout - international interna...,- keep up with the vibrant services given to e...,One of the best MNC company in Singapore!- ver...
2,Business Specialist,An amazing opportunity. So many talented co-wo...,There really isn't anything negative to point ...,Keep up the great work. I learned so much and ...,Business SpecialistAn amazing opportunity. So ...
3,Fast Paced Environment where you learn a lot,"The upward mobility, fast paced, growth and te...",Amazon is the gift that takes. Ensure you have...,Allow your leaders in the fulfillment centers ...,Fast Paced Environment where you learn a lotTh...
4,Overview,"High Salary, Benefits, Raises yearly.",Short Breaks. Unattainable Metrics. Dealing wi...,Treating everyone equally. Don't Play Favorites,"OverviewHigh Salary, Benefits, Raises yearly.S..."
...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,as of me nothing we can enjoy while skiping work,good at all,good timefeel free environment good infrastruc...
36657,good,it's a good place to start one's career,the pay check may not that good,NaN,goodit's a good place to start one's careerthe...
36658,Consumer Product Advisor,"Meeting tons of people, fun work environment, ...","Hard area of growth, work full time hours but ...",Try to help for growth in a more open way,Consumer Product AdvisorMeeting tons of people...
36659,Fulltime,Offers of Great Opportunity and good Benefits,Work is competing process sa me employee don’t...,Don’t put the work in Rating system,FulltimeOffers of Great Opportunity and good B...


In [57]:
y_train_all_copy

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,1,1,2,1,1
1,2,2,1,2,1,1
2,2,2,2,1,1,2
3,0,2,2,2,1,2
4,1,0,0,2,1,1
...,...,...,...,...,...,...
36656,2,2,2,2,2,2
36657,2,1,2,2,1,1
36658,1,2,0,2,1,1
36659,2,2,2,2,2,2


## Prepare Features

In [58]:
preprocessor = Preprocessor()

In [60]:
X_train_all_copy = preprocessor.fit_transform(X_train_all_copy)

X_test_all_copy = preprocessor.fit_transform(X_test_all_copy)

In [61]:
feature_engineering = FeatureEngineer()

In [62]:
X_train_all_copy = feature_engineering.fit_transform(X_train_all_copy)

100%|██████████| 5/5 [01:43<00:00, 20.61s/it]


In [63]:
X_train_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,0.5,0.5,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,0.3,1.0,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,0.0,0.0,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,0.6,0.2,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,0.0,0.0,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,0.6,0.7,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,4,27,18,0,56,0.6,0.7,0.600000,0.700000,0.600000,0.700000,NaN,NaN,0.600000,0.700000
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,0.0,0.0,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,0.0,0.0,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [64]:
X_test_all_copy = feature_engineering.fit_transform(X_test_all_copy)

100%|██████████| 5/5 [00:45<00:00,  9.01s/it]


In [65]:
X_train_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,0.5,0.5,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,0.3,1.0,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,0.0,0.0,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,0.6,0.2,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,0.0,0.0,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,0.6,0.7,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,4,27,18,0,56,0.6,0.7,0.600000,0.700000,0.600000,0.700000,NaN,NaN,0.600000,0.700000
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,0.0,0.0,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,0.0,0.0,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [66]:
X_test_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,overall satisfying experience,great people infact meet really brilliant peop...,switching team within microsoft insanely hard ...,NaN,overall satisfying experiencegreat people infa...,29,173,97,0,300,0.500,0.25000,0.700000,0.700000,0.447222,-0.063889,NaN,NaN,0.565741,0.334259
1,good training experience,job get back workforce,place consider long term employment training u...,stop rushing expecting unrealistic result litt...,good training experienceit job get back workfo...,24,22,109,53,216,0.600,0.70000,0.000000,0.000000,0.500000,0.075000,0.750000,-0.343750,0.516667,0.027083
2,great company known worth,overall unarguably fantastic place work,little focus retention career development outs...,NaN,great company known worthoverall unarguably fa...,25,39,145,0,210,0.425,0.55000,0.450000,0.200000,0.350000,-0.062500,NaN,NaN,0.540000,0.202500
3,good choice long term career,work pay consistant,treated valued employee rather expendable part...,NaN,good choice long term careerwork pay consistan...,28,19,54,0,105,0.500,0.32500,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.500000,0.325000
4,part time fulfillment associate,worked warehouse one say amazon build center e...,beginning house hr seem available gotten bette...,management help come scanning,part time fulfillment associatei worked wareho...,31,332,135,29,533,0.000,0.00000,0.341667,0.112500,0.300000,0.300000,0.000000,0.000000,0.327778,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,non stimulating job,looking job requires minimal thinking salary g...,begin gon na start rambling first hour awful h...,system work technically yes terrible one emplo...,non stimulating job looking job requires minim...,19,835,1412,88,2357,0.000,0.00000,0.373106,0.084659,0.424180,0.023348,0.550000,-0.500000,0.411542,0.027727
15709,work ummm,easy job nice people work great manager alot o...,part time alot hour boring alot sent home early,NaN,work ummmeasy job nice people work great manag...,9,73,47,0,137,0.000,0.00000,0.833333,0.658333,0.650000,-0.450000,NaN,NaN,0.760000,0.260000
15710,company huge focus customer little priority em...,employee kind one another willing help know un...,lack training management stress deal,better training would lot,company huge focus customer little priority em...,52,55,36,25,171,0.700,0.10625,0.825000,0.425000,0.000000,0.000000,0.500000,0.500000,0.710000,0.312500
15711,internship experience,really fun work really skill learning experien...,thing like difficulty work expect great big co...,maybe hire people ease workload everyone else ...,internship experienceit really fun work really...,21,96,51,82,255,0.000,0.00000,0.565000,0.480000,0.425000,0.400000,0.683333,0.250000,0.572500,0.395000


In [26]:
X_train_all_copy.dropna(inplace=True)

In [27]:
X_train_all_copy.reset_index(drop=True, inplace=True)

In [28]:
X_test_all_copy.dropna(inplace=True)

In [29]:
X_test_all_copy.reset_index(drop=True, inplace=True)

In [30]:
X_train_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,0.5,0.50,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,0.3,1.00,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,0.0,0.00,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,0.6,0.20,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,0.0,0.00,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20918,good upper management,great benefit insurance pet insurance party te...,coming startup overwhelmed lack flexibility ov...,simplify,good upper management great benefit insurance ...,21,100,101,8,232,0.3,0.35,0.640422,0.180519,0.000000,0.000000,0.000000,0.000000,0.526948,0.237013
20919,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,0.6,0.70,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
20920,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,0.0,0.00,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
20921,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,0.0,0.00,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [41]:
X_test_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,good training experience,job get back workforce,place consider long term employment training u...,stop rushing expecting unrealistic result litt...,good training experienceit job get back workfo...,24,22,109,53,216,0.6,0.70000,0.000000,0.000000,0.50000,0.075000,0.750000,-0.343750,0.516667,0.027083
1,part time fulfillment associate,worked warehouse one say amazon build center e...,beginning house hr seem available gotten bette...,management help come scanning,part time fulfillment associatei worked wareho...,31,332,135,29,533,0.0,0.00000,0.341667,0.112500,0.30000,0.300000,0.000000,0.000000,0.327778,0.175000
2,part time specialist,great staff manager care getting want get care...,long line get want get apple want management m...,open employee idea,part time specialistgreat staff manager care g...,20,86,170,18,296,0.0,0.00000,0.650000,0.575000,0.47500,0.262500,0.500000,0.000000,0.491667,0.233333
3,time apple,apple well run organized company start made fr...,apart home agent program expanded southeast pa...,increase salary home agent,time appleapple well run organized company sta...,10,107,57,26,202,0.0,0.00000,0.518750,0.187500,0.00000,0.000000,0.000000,0.000000,0.518750,0.187500
4,engineer,love amazing perk mostly food keep going day b...,competitive environment high paced hey valley,nothing add time,engineerlove amazing perk mostly food keep goi...,8,57,45,16,127,0.0,0.00000,0.750000,0.575000,0.54000,0.160000,0.000000,0.000000,0.735000,0.490000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8963,good place developer,bright folk big product complex problem space,review system difficulty understanding promoti...,ditch review system,good place developer bright folk big product c...,20,45,109,19,196,0.6,0.70000,0.433333,0.133333,0.37500,0.250000,0.000000,0.000000,0.425000,0.262500
8964,non stimulating job,looking job requires minimal thinking salary g...,begin gon na start rambling first hour awful h...,system work technically yes terrible one emplo...,non stimulating job looking job requires minim...,19,835,1412,88,2357,0.0,0.00000,0.373106,0.084659,0.42418,0.023348,0.550000,-0.500000,0.411542,0.027727
8965,company huge focus customer little priority em...,employee kind one another willing help know un...,lack training management stress deal,better training would lot,company huge focus customer little priority em...,52,55,36,25,171,0.7,0.10625,0.825000,0.425000,0.00000,0.000000,0.500000,0.500000,0.710000,0.312500
8966,internship experience,really fun work really skill learning experien...,thing like difficulty work expect great big co...,maybe hire people ease workload everyone else ...,internship experienceit really fun work really...,21,96,51,82,255,0.0,0.00000,0.565000,0.480000,0.42500,0.400000,0.683333,0.250000,0.572500,0.395000


In [68]:
df_train_all = pd.concat([X_train_all_copy, y_train_all_copy], axis=1)
df_test_all = pd.concat([X_test_all_copy, y_test_all_copy], axis=1)

In [36]:
df_train_all.dropna(inplace=True)

In [38]:
df_test_all.dropna(inplace=True)

In [69]:
display(df_train_all)
display(df_test_all)

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,...,0.000000,0.000000,0.533333,0.400000,1,1,1,2,1,1
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,...,0.333333,0.166667,0.205556,0.294444,2,2,1,2,1,1
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,...,0.837500,0.725000,0.626364,0.335152,2,2,2,1,1,2
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,...,0.322727,0.063636,0.452564,0.057459,0,2,2,2,1,2
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,...,0.625000,0.250000,0.516667,0.166667,1,0,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,...,0.600000,0.700000,0.625000,0.550000,2,2,2,2,2,2
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,4,27,18,0,56,...,NaN,NaN,0.600000,0.700000,2,1,2,2,1,1
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,...,0.500000,0.000000,0.440000,0.350000,1,2,0,2,1,1
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,...,0.000000,0.000000,0.675000,0.750000,2,2,2,2,2,2


,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,overall satisfying experience,great people infact meet really brilliant peop...,switching team within microsoft insanely hard ...,NaN,overall satisfying experiencegreat people infa...,29,173,97,0,300,...,NaN,NaN,0.565741,0.334259,2,1,2,2,1,1
1,good training experience,job get back workforce,place consider long term employment training u...,stop rushing expecting unrealistic result litt...,good training experienceit job get back workfo...,24,22,109,53,216,...,0.750000,-0.343750,0.516667,0.027083,1,0,0,0,0,0
2,great company known worth,overall unarguably fantastic place work,little focus retention career development outs...,NaN,great company known worthoverall unarguably fa...,25,39,145,0,210,...,NaN,NaN,0.540000,0.202500,1,2,1,2,2,1
3,good choice long term career,work pay consistant,treated valued employee rather expendable part...,NaN,good choice long term careerwork pay consistan...,28,19,54,0,105,...,NaN,NaN,0.500000,0.325000,0,0,0,0,0,0
4,part time fulfillment associate,worked warehouse one say amazon build center e...,beginning house hr seem available gotten bette...,management help come scanning,part time fulfillment associatei worked wareho...,31,332,135,29,533,...,0.000000,0.000000,0.327778,0.175000,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,non stimulating job,looking job requires minimal thinking salary g...,begin gon na start rambling first hour awful h...,system work technically yes terrible one emplo...,non stimulating job looking job requires minim...,19,835,1412,88,2357,...,0.550000,-0.500000,0.411542,0.027727,0,1,0,2,0,1
15709,work ummm,easy job nice people work great manager alot o...,part time alot hour boring alot sent home early,NaN,work ummmeasy job nice people work great manag...,9,73,47,0,137,...,NaN,NaN,0.760000,0.260000,1,2,1,1,2,0
15710,company huge focus customer little priority em...,employee kind one another willing help know un...,lack training management stress deal,better training would lot,company huge focus customer little priority em...,52,55,36,25,171,...,0.500000,0.500000,0.710000,0.312500,0,0,2,1,0,0
15711,internship experience,really fun work really skill learning experien...,thing like difficulty work expect great big co...,maybe hire people ease workload everyone else ...,internship experienceit really fun work really...,21,96,51,82,255,...,0.683333,0.250000,0.572500,0.395000,2,2,2,2,2,2


In [70]:
X_train_all_copy#.dtype

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,0.5,0.5,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,0.3,1.0,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,0.0,0.0,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,0.6,0.2,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,0.0,0.0,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,0.6,0.7,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,4,27,18,0,56,0.6,0.7,0.600000,0.700000,0.600000,0.700000,NaN,NaN,0.600000,0.700000
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,0.0,0.0,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,0.0,0.0,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [71]:
y_train_all_copy#['overall'].dtype

,work-balance,culture-values,career-opportunities,comp-benefits,senior-mgmt,overall
0,1,1,1,2,1,1
1,2,2,1,2,1,1
2,2,2,2,1,1,2
3,0,2,2,2,1,2
4,1,0,0,2,1,1
...,...,...,...,...,...,...
36656,2,2,2,2,2,2
36657,2,1,2,2,1,1
36658,1,2,0,2,1,1
36659,2,2,2,2,2,2


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from textblob import TextBlob
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from tqdm import tqdm
from sklearn.naive_bayes import MultinomialNB

In [74]:
'''scores_dict= {}
for score in y_test_all_copy.columns:
    result_scores = {}
    for feature in df_train_all.select_dtypes('object').columns:
        # instantiate vectorizer
        vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1,7))
        # vectorize train set
        X_train_all_copy = vectorizer.fit_transform(df_train_all[feature].astype('U'))
        # vectorize test set
        X_test_all_copy = vectorizer.transform(df_test_all[feature].astype('U'))
        # select target
        target = y_test_all_copy[score]
        # instantiate model
        model = MultinomialNB()
        # fit model
        model.fit(X_train_all_copy, y_train_all_copy[score])
        # predict with train and test sets and store predictions in new col
        df_train_all[f"{feature}_{score}_nb"] = model.predict(X_train_all_copy)
        df_test_all[f"{feature}_{score}_nb"] = model.predict(X_test_all_copy)
        # evaluate model and append results in dictionary
        result_scores[f'{feature}'] = model.score(X_test_all_copy, target)
    # append results to scores dictionary
    scores_dict[f'{score}'] = result_scores'''

'scores_dict= {}\nfor score in y_test_all_copy.columns:\n    result_scores = {}\n    for feature in df_train_all.select_dtypes(\'object\').columns:\n        # instantiate vectorizer\n        vectorizer = TfidfVectorizer(analyzer=\'char\', ngram_range=(1,7))\n        # vectorize train set\n        X_train_all_copy = vectorizer.fit_transform(df_train_all[feature].astype(\'U\'))\n        # vectorize test set\n        X_test_all_copy = vectorizer.transform(df_test_all[feature].astype(\'U\'))\n        # select target\n        target = y_test_all_copy[score]\n        # instantiate model\n        model = MultinomialNB()\n        # fit model\n        model.fit(X_train_all_copy, y_train_all_copy[score])\n        # predict with train and test sets and store predictions in new col\n        df_train_all[f"{feature}_{score}_nb"] = model.predict(X_train_all_copy)\n        df_test_all[f"{feature}_{score}_nb"] = model.predict(X_test_all_copy)\n        # evaluate model and append results in dictionary\

In [73]:
df_train_all, df_test_all, scores_dict = add_multinomial_nb_prediction_feature(df_train_all, df_test_all, y_train_all_copy, y_test_all_copy)

In [183]:
df_train_all

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,summary_senior-mgmt_nb,positives_senior-mgmt_nb,negatives_senior-mgmt_nb,advice_to_mgmt_senior-mgmt_nb,review_senior-mgmt_nb,summary_overall_nb,positives_overall_nb,negatives_overall_nb,advice_to_mgmt_overall_nb,review_overall_nb
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,32,46,40,180,300,...,2,2,2,2,2,2,2,2,2,2
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,30,105,52,35,225,...,2,2,2,2,2,2,2,2,2,2
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,19,112,72,115,323,...,2,2,2,2,2,2,2,2,2,2
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,32,199,200,92,529,...,2,2,2,0,2,2,2,2,2,2
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,8,32,59,39,140,...,2,2,2,0,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,9,57,26,4,99,...,2,2,2,2,2,2,2,2,2,2
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,4,27,18,0,56,...,2,2,2,2,2,2,2,2,2,2
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,24,69,51,24,173,...,2,2,2,2,2,2,2,2,2,2
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,8,36,45,24,119,...,2,2,2,2,2,2,2,2,2,2


In [76]:
df_test_all

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,...,summary_senior-mgmt_nb,positives_senior-mgmt_nb,negatives_senior-mgmt_nb,advice_to_mgmt_senior-mgmt_nb,review_senior-mgmt_nb,summary_overall_nb,positives_overall_nb,negatives_overall_nb,advice_to_mgmt_overall_nb,review_overall_nb
0,overall satisfying experience,great people infact meet really brilliant peop...,switching team within microsoft insanely hard ...,NaN,overall satisfying experiencegreat people infa...,29,173,97,0,300,...,2,2,2,2,2,2,2,2,2,2
1,good training experience,job get back workforce,place consider long term employment training u...,stop rushing expecting unrealistic result litt...,good training experienceit job get back workfo...,24,22,109,53,216,...,2,2,2,0,2,2,2,2,2,2
2,great company known worth,overall unarguably fantastic place work,little focus retention career development outs...,NaN,great company known worthoverall unarguably fa...,25,39,145,0,210,...,2,2,2,2,2,2,2,2,2,2
3,good choice long term career,work pay consistant,treated valued employee rather expendable part...,NaN,good choice long term careerwork pay consistan...,28,19,54,0,105,...,2,2,0,2,2,2,2,2,2,2
4,part time fulfillment associate,worked warehouse one say amazon build center e...,beginning house hr seem available gotten bette...,management help come scanning,part time fulfillment associatei worked wareho...,31,332,135,29,533,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,non stimulating job,looking job requires minimal thinking salary g...,begin gon na start rambling first hour awful h...,system work technically yes terrible one emplo...,non stimulating job looking job requires minim...,19,835,1412,88,2357,...,2,2,2,0,2,2,2,2,2,2
15709,work ummm,easy job nice people work great manager alot o...,part time alot hour boring alot sent home early,NaN,work ummmeasy job nice people work great manag...,9,73,47,0,137,...,2,2,2,2,2,2,2,2,2,2
15710,company huge focus customer little priority em...,employee kind one another willing help know un...,lack training management stress deal,better training would lot,company huge focus customer little priority em...,52,55,36,25,171,...,2,2,2,2,2,2,2,2,2,2
15711,internship experience,really fun work really skill learning experien...,thing like difficulty work expect great big co...,maybe hire people ease workload everyone else ...,internship experienceit really fun work really...,21,96,51,82,255,...,2,2,2,0,2,2,2,2,2,2


In [83]:
#df_test.dropna(inplace=True)

In [84]:
scores_dict

{'work-balance': {'summary': 0.5282886781645771,
  'positives': 0.5004136702093808,
  'negatives': 0.5312798319862534,
  'advice_to_mgmt': 0.5274613377458156,
  'review': 0.4965951759689429},
 'culture-values': {'summary': 0.6560173105072233,
  'positives': 0.6483803220263477,
  'negatives': 0.6482530388849996,
  'advice_to_mgmt': 0.6488258130210653,
  'review': 0.6481257557436517},
 'career-opportunities': {'summary': 0.6036402978425508,
  'positives': 0.5975307070578502,
  'negatives': 0.5974034239165023,
  'advice_to_mgmt': 0.5975307070578502,
  'review': 0.5971488576338064},
 'comp-benefits': {'summary': 0.7078851906065041,
  'positives': 0.7080124737478521,
  'negatives': 0.7080124737478521,
  'advice_to_mgmt': 0.7080124737478521,
  'review': 0.7080124737478521},
 'senior-mgmt': {'summary': 0.5437535798383504,
  'positives': 0.5099599058104755,
  'negatives': 0.5344619105199516,
  'advice_to_mgmt': 0.5695284159613059,
  'review': 0.5016865016228601},
 'overall': {'summary': 0.5181

In [110]:
scaler = CustomMinMaxScaler()

X_train_all_copy_scaled, X_test_all_copy_scaled = scaler.fit_transform(X_train_all_copy, X_test_all_copy)

ValueError: at least one array or dtype is required

In [103]:
X_train_all_copy_scaled

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,0.293578,0.012004,0.004619,0.060221,0.031955,0.5,0.5,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,0.275229,0.027401,0.006005,0.011710,0.023365,0.3,1.0,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,0.174312,0.029228,0.008314,0.038474,0.034589,0.0,0.0,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,0.293578,0.051931,0.023095,0.030780,0.058183,0.6,0.2,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,0.073394,0.008351,0.006813,0.013048,0.013630,0.0,0.0,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,0.082569,0.014875,0.003002,0.001338,0.008934,0.6,0.7,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,0.036697,0.007046,0.002079,0.000000,0.004009,0.6,0.7,0.600000,0.700000,0.600000,0.700000,NaN,NaN,0.600000,0.700000
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,0.220183,0.018006,0.005889,0.008029,0.017409,0.0,0.0,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,0.073394,0.009395,0.005196,0.008029,0.011224,0.0,0.0,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [107]:
X_train_all_copy

,summary,positives,negatives,advice_to_mgmt,review,summary_length,positives_length,negatives_length,advice_to_mgmt_length,review_length,subjectivity_summary,polarity_summary,subjectivity_positives,polarity_positives,subjectivity_negatives,polarity_negatives,subjectivity_advice_to_mgmt,polarity_advice_to_mgmt,subjectivity_review,polarity_review
0,many microsofts within microsoft,smart colleague flexible schedule good benefit,review model foster enough collaboration,please work foster collaboration within across...,many microsofts within microsoftsmart colleagu...,0.293578,0.012004,0.004619,0.060221,0.031955,0.5,0.5,0.621429,0.457143,0.500000,0.000000,0.000000,0.000000,0.533333,0.400000
1,one best mnc company singapore,responsive welfare cover area provide training...,lacking bonus payout international internal tr...,keep vibrant service given employee,one best mnc company singapore responsive welf...,0.275229,0.027401,0.006005,0.011710,0.023365,0.3,1.0,0.300000,0.300000,0.000000,0.000000,0.333333,0.166667,0.205556,0.294444
2,business specialist,amazing opportunity many talented co worker sh...,really anything negative point many positive t...,keep great work learned much enjoyed part amaz...,business specialistan amazing opportunity many...,0.174312,0.029228,0.008314,0.038474,0.034589,0.0,0.0,0.666667,0.300000,0.409091,0.065455,0.837500,0.725000,0.626364,0.335152
3,fast paced environment learn lot,upward mobility fast paced growth technology a...,amazon gift take ensure strict boundary much w...,allow leader fulfillment center time develop t...,fast paced environment learn lotthe upward mob...,0.293578,0.051931,0.023095,0.030780,0.058183,0.6,0.2,0.501136,0.109091,0.438889,-0.002778,0.322727,0.063636,0.452564,0.057459
4,overview,high salary benefit raise yearly,short break unattainable metric dealing cowork...,treating everyone equally play favorite,overviewhigh salary benefit raise yearly short...,0.073394,0.008351,0.006813,0.013048,0.013630,0.0,0.0,0.540000,0.160000,0.300000,0.000000,0.625000,0.250000,0.516667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36656,good time,feel free environment good infrastructure well...,nothing enjoy skiping work,good,good timefeel free environment good infrastruc...,0.082569,0.014875,0.003002,0.001338,0.008934,0.6,0.7,0.700000,0.550000,0.500000,0.400000,0.600000,0.700000,0.625000,0.550000
36657,good,good place start one career,pay check may good,NaN,goodit good place start one careerthe pay chec...,0.036697,0.007046,0.002079,0.000000,0.004009,0.6,0.7,0.600000,0.700000,0.600000,0.700000,NaN,NaN,0.600000,0.700000
36658,consumer product advisor,meeting ton people fun work environment fun co...,hard area growth work full time hour paid part...,try help growth open way,consumer product advisormeeting ton people fun...,0.220183,0.018006,0.005889,0.008029,0.017409,0.0,0.0,0.383333,0.466667,0.545833,0.029167,0.500000,0.000000,0.440000,0.350000
36659,fulltime,offer great opportunity good benefit,work competing process sa employee ’ give way,’ put work rating system,fulltimeoffers great opportunity good benefits...,0.073394,0.009395,0.005196,0.008029,0.011224,0.0,0.0,0.675000,0.750000,0.000000,0.000000,0.000000,0.000000,0.675000,0.750000


In [106]:
X_test_all_copy_scaled = scaler.transform(X_test_all_copy)

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [89]:
from WorkforceSentimentMonitoring.encoders import PredictionFeaturesExtractor

In [90]:
#extractor = PredictionFeaturesExtractor()

In [ ]:
#vec = extractor.vectorize(FEATURE_COLS)

In [92]:
#X_train_all_copy_final = extractor.fit_transform(X_train_all_copy_scaled, y_train_copy)

In [ ]:
get_linear_regression_cols(X_train_all_copy_scaled, X_test, y_train)

# Use trained model

In [80]:
# pipelines for preprocesser, polarity&subjectivity, nb_feature_engineer_model for each new columns
# interate rows of new dataframe to create new columns '.._..._nb'
# return a new dataframe with new features

In [1]:
#tbc